In [32]:
import os
os.chdir('C:/Users/RH/A_try_change/ESM320_clean/test_data')

In [33]:
import pandas as pd
import numpy as np
import esm
import torch

print(pd.__version__)
print(np.__version__)
print(esm.__version__)
print(torch.__version__)

1.5.3
1.22.0
2.0.0
2.0.0


In [34]:
import numpy as np
import pandas as pd

In [41]:
#training dataset loading
#打开指定路径的excel文件
excel_path = r'C:\Users\RH\A_try_change\ESM320_clean\test_data\X_test_filter.xlsx'
dataset = pd.read_excel(excel_path, na_filter=False)
sequence_list = dataset['Sequence']

In [42]:
print(sequence_list)

0             GLFDIIKNIFSGL
1                 FLGALFKVA
2             IFSAIAGLLSNLL
3             FLGALFKALSKLL
4         LLNQELLLNPTHQIYPV
5                  GLLWHLLH
6             FFPVIGRILNGIL
7             TPFLLVGTQIDLR
8              SKWMSLLKHILK
9                GLLSRIKTLL
10            FLGALFKVASKVL
11            LLAIVGNLLKSLL
12            FLPLLASLFSGLF
13                  GKLWLKG
14            LGALFKVASKVLP
15             IKWAGKLFKLFK
16           FSLIPSAIGGLISA
17            FLPLVTGLLSGLL
18            FLPLLASLFSRLL
19            FLKGIIDTVSNWL
20                GFIFHIIKG
21            SILPTIVSFLSKF
22          VNFLLHKIYGNIRYS
23             FVQWFSKFLGRI
24               GLLKIIKTLL
25             LLPIVGNLLNSL
26             GMWSKILPHLIR
27              GFFALIPKIIS
28            FLGALAKALSKLL
29               GLLKNIKTLL
30            FFPFVAKLLKGLF
31            LLPIVGNLLNSLL
32             GMWSKILGPLIR
33        MWITNGGVANWYFVLAR
34    FLISIPYSASIGGTATLTGTA
35        GLLDFLKAAG

In [43]:
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence)
print(peptide_sequence_list)

[('GLFDIIKNIFSGL', 'GLFDIIKNIFSGL'), ('FLGALFKVA', 'FLGALFKVA'), ('IFSAIAGLLSNLL', 'IFSAIAGLLSNLL'), ('FLGALFKALSKLL', 'FLGALFKALSKLL'), ('LLNQELLLNPTHQIYPV', 'LLNQELLLNPTHQIYPV'), ('GLLWHLLH', 'GLLWHLLH'), ('FFPVIGRILNGIL', 'FFPVIGRILNGIL'), ('TPFLLVGTQIDLR', 'TPFLLVGTQIDLR'), ('SKWMSLLKHILK', 'SKWMSLLKHILK'), ('GLLSRIKTLL', 'GLLSRIKTLL'), ('FLGALFKVASKVL', 'FLGALFKVASKVL'), ('LLAIVGNLLKSLL', 'LLAIVGNLLKSLL'), ('FLPLLASLFSGLF', 'FLPLLASLFSGLF'), ('GKLWLKG', 'GKLWLKG'), ('LGALFKVASKVLP', 'LGALFKVASKVLP'), ('IKWAGKLFKLFK', 'IKWAGKLFKLFK'), ('FSLIPSAIGGLISA', 'FSLIPSAIGGLISA'), ('FLPLVTGLLSGLL', 'FLPLVTGLLSGLL'), ('FLPLLASLFSRLL', 'FLPLLASLFSRLL'), ('FLKGIIDTVSNWL', 'FLKGIIDTVSNWL'), ('GFIFHIIKG', 'GFIFHIIKG'), ('SILPTIVSFLSKF', 'SILPTIVSFLSKF'), ('VNFLLHKIYGNIRYS', 'VNFLLHKIYGNIRYS'), ('FVQWFSKFLGRI', 'FVQWFSKFLGRI'), ('GLLKIIKTLL', 'GLLKIIKTLL'), ('LLPIVGNLLNSL', 'LLPIVGNLLNSL'), ('GMWSKILPHLIR', 'GMWSKILPHLIR'), ('GFFALIPKIIS', 'GFFALIPKIIS'), ('FLGALAKALSKLL', 'FLGALAKALSKLL'), ('GLL

In [44]:
#调试完毕，以此为准！
import torch
import esm
import pandas as pd

#load ESM-2 model
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
    
#load data
data = peptide_sequence_list
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[6], return_contacts=True)
token_representations = results["representations"][6]
    
# Generate per-sequence representations via averaging
sequence_representations = []
for i, token_len in enumerate(batch_lens):
    each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
    sequence_representations.append(each_seq_rep)

embedding_results = pd.DataFrame(sequence_representations)
print(embedding_results)
embedding_results.to_csv('X_filter.csv')

         0         1         2         3         4         5         6    \
0   0.011622  0.044379  0.094179  0.028783 -0.139837 -0.269270  0.113180   
1   0.009907 -0.031947  0.285402  0.066398  0.074255 -0.143199 -0.147464   
2   0.120423 -0.021647  0.161637  0.005580  0.018811 -0.186077 -0.086914   
3  -0.132438 -0.060827  0.175465  0.106694  0.046132 -0.178985 -0.153216   
4   0.062694 -0.361747  0.327823  0.117543  0.070681 -0.260072 -0.038333   
5  -0.075202 -0.284899  0.259774  0.177632  0.070193 -0.424365 -0.256857   
6   0.052943 -0.035889  0.270819  0.216553 -0.015643 -0.305098 -0.136220   
7   0.125369 -0.289523  0.289406  0.084858  0.117776 -0.205162 -0.155635   
8  -0.051633 -0.297644  0.229220  0.233153  0.014019 -0.296500 -0.170733   
9   0.057561 -0.312799  0.219698  0.208805  0.092294 -0.301053 -0.184224   
10 -0.063212  0.096035  0.250879  0.115816  0.021990 -0.131530 -0.139505   
11  0.072913 -0.031585  0.295808  0.122115  0.041929 -0.241224 -0.175609   
12 -0.092326